<p style="font-family: 'Courier New', Courier, monospace; font-size: 30px; font-weight: bold; color: blue; text-align: left;">
Recurrent Neural Networks (RNN) - One-Way Modeling
</p>

In [1]:
# Libraries for data manipulation and visualization
import numpy as np                               # For numerical operations
import pandas as pd                              # For data manipulation
import matplotlib.pyplot as plt                  # For plotting
import seaborn as sns                            # For advanced data visualization

# Libraries for model building and training
import tensorflow as tf                          # For deep learning framework
from keras.models import Sequential              # For creating sequential models
from keras.layers import Dense, Input, BatchNormalization, Dropout, LeakyReLU, SimpleRNN  # Layers for building neural networks
from keras.regularizers import l2               # For L2 regularization
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau  # For training optimization

# Libraries for evaluation and preprocessing
from sklearn.metrics import (                   # For model evaluation metrics
    mean_squared_error, 
    r2_score, 
    mean_absolute_percentage_error, 
    median_absolute_error
)
from sklearn.preprocessing import StandardScaler # For standardizing features
from sklearn.model_selection import KFold, train_test_split, PredefinedSplit # For cross-validation and splitting data

# Set seed for reproducibility
GLOBAL_SEED = 50
np.random.seed(GLOBAL_SEED)                      # Seed for NumPy
tf.random.set_seed(GLOBAL_SEED)                  # Seed for TensorFlow

2026-01-07 13:12:43.910689: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-01-07 13:12:43.932530: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
# Time-aware data load (from Data Preparation.ipynb outputs)

base_path = '../../Comprehensive ML - Files & Plots etc'

df_train = pd.read_csv(f"{base_path}/train.csv", parse_dates=['time'])
df_test  = pd.read_csv(f"{base_path}/test.csv", parse_dates=['time'])
fold_assignments = np.load(f"{base_path}/train_folds.npy")

feature_names = [
    'distance', 'frequency', 'c_walls', 'w_walls', 'co2', 'humidity',
    'pm25', 'pressure', 'temperature', 'snr'
]

X_train = df_train[feature_names].to_numpy()
y_train = df_train['PL'].to_numpy()
X_test  = df_test[feature_names].to_numpy()
y_test  = df_test['PL'].to_numpy()

time_train = df_train['time'].to_numpy()
time_test  = df_test['time'].to_numpy()

ps = PredefinedSplit(fold_assignments)  # reuse the time-aware folds

print(f"Train: {len(df_train)} rows, Test: {len(df_test)} rows")
print(f"Train window: {df_train.time.min()} -> {df_train.time.max()}")
print(f"Test window:  {df_test.time.min()} -> {df_test.time.max()}")

FileNotFoundError: [Errno 2] No such file or directory: '../../Comprehensive ML - Files & Plots etc/train.csv'

In [ ]:
# use a fraction of the data (keeps time order) *per fold*. Set USE_SUBSET=False for full data.
USE_SUBSET = True
DATA_FRACTION = 0.25  # fraction of data to keep from each fold

if USE_SUBSET:
    fold_ids = np.unique(fold_assignments[fold_assignments != -1])
    keep_mask = np.zeros(len(df_train), dtype=bool)
    kept_counts = {}

    # keep the first DATA_FRACTION portion within each fold, preserving order
    for fid in fold_ids:
        idx = np.flatnonzero(fold_assignments == fid)
        keep_n = max(1, int(np.ceil(len(idx) * DATA_FRACTION)))
        keep_mask[idx[:keep_n]] = True
        kept_counts[fid] = keep_n

    # (optional) if there are -1 labels in train, subset them too
    if np.any(fold_assignments == -1):
        idx = np.flatnonzero(fold_assignments == -1)
        keep_n = max(1, int(np.ceil(len(idx) * DATA_FRACTION)))
        keep_mask[idx[:keep_n]] = True
        kept_counts['-1'] = keep_n

    df_train = df_train.iloc[keep_mask].copy()
    fold_assignments = fold_assignments[keep_mask]
    # subset test set separately if you still want a fraction
    n_test = max(1, int(np.ceil(len(df_test) * DATA_FRACTION)))
    df_test = df_test.iloc[:n_test].copy()

    print("Using subset per fold:", kept_counts)
    print(f"Train rows kept: {len(df_train)}, Test rows kept: {len(df_test)}")
else:
    print("Using full dataset")

if len(df_train) == 0 or len(df_test) == 0:
    raise ValueError("Subset produced empty data; increase DATA_FRACTION or disable USE_SUBSET.")

# refresh arrays/split objects
X_train = df_train[feature_names].to_numpy()
y_train = df_train["PL"].to_numpy()
X_test = df_test[feature_names].to_numpy()
y_test = df_test["PL"].to_numpy()
time_train = df_train["time"].to_numpy()
time_test = df_test["time"].to_numpy()
ps = PredefinedSplit(fold_assignments)

fold_ids = fold_assignments[fold_assignments != -1]
print("Unique CV fold labels:", np.unique(fold_ids), "count:", np.unique(fold_ids).size)
print("Counts per fold:", np.bincount(fold_ids) if fold_ids.size else "none")

In [ ]:
# Scale features (fit on train only)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Sequence length for RNN
SEQ_LEN = 10

def make_sequences(X, y, seq_len):
    Xs, ys = [], []
    for i in range(len(X) - seq_len + 1):
        Xs.append(X[i:i + seq_len])
        ys.append(y[i + seq_len - 1])
    return np.array(Xs), np.array(ys)

def build_sequences_by_fold(X, y, fold_assignments, seq_len):
    X_list, y_list, fold_list = [], [], []
    for fid in np.unique(fold_assignments):
        idx = np.flatnonzero(fold_assignments == fid)
        if idx.size < seq_len:
            continue
        X_fold = X[idx]
        y_fold = y[idx]
        X_seq, y_seq = make_sequences(X_fold, y_fold, seq_len)
        X_list.append(X_seq)
        y_list.append(y_seq)
        fold_list.append(np.full(len(y_seq), fid, dtype=int))
    if not X_list:
        raise ValueError("No sequences created; reduce SEQ_LEN or check fold sizes.")
    return np.concatenate(X_list), np.concatenate(y_list), np.concatenate(fold_list)

# Build sequences for training (respecting fold boundaries)
X_train_seq, PL_train_seq, fold_seq = build_sequences_by_fold(
    X_train_scaled, y_train, fold_assignments, SEQ_LEN
)

# Build sequences for test set
if len(X_test_scaled) < SEQ_LEN:
    raise ValueError("Test set shorter than SEQ_LEN; reduce SEQ_LEN or disable subset.")
X_test_seq, PL_test_seq = make_sequences(X_test_scaled, y_test, SEQ_LEN)

# PredefinedSplit for sequence-level CV
ps_seq = PredefinedSplit(fold_seq)

print(f"Train sequences: {X_train_seq.shape}, Test sequences: {X_test_seq.shape}")
print("Sequence fold labels:", np.unique(fold_seq))

In [ ]:
def create_rnn_model(seq_len, n_features):
    """Creates a one-way RNN model for regression with regularization."""
    model = Sequential()

    # Explicit Input layer
    model.add(Input(shape=(seq_len, n_features)))

    # One-way RNN layer
    model.add(SimpleRNN(32, activation='tanh', kernel_regularizer=l2(0.001)))
    model.add(Dropout(0.2))

    # Dense block
    model.add(Dense(16, kernel_regularizer=l2(0.001)))
    model.add(LeakyReLU(negative_slope=0.1))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))

    # Output layer for regression
    model.add(Dense(1, activation='linear'))

    # Compile the model
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
    return model

# Instantiate the model
seq_len = X_train_seq.shape[1]
n_features = X_train_seq.shape[2]
model = create_rnn_model(seq_len, n_features)
model.summary()

# Define Callbacks
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=30,
    restore_best_weights=True
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=10,
    min_lr=1e-6,
    verbose=1
)

# Train the model with Callbacks
history = model.fit(
    X_train_seq,
    PL_train_seq,
    validation_split=0.2,
    epochs=500,
    batch_size=128,
    verbose=1,
    callbacks=[early_stop, reduce_lr],
    shuffle=False
)

print("\nModel training completed...\n")

In [ ]:
# Evaluate the trained model on training and test data and display metrics in a table

# Evaluate the model on training data
train_loss, train_mae = model.evaluate(X_train_seq, PL_train_seq, verbose=0)

# Evaluate the model on test data
test_loss, test_mae = model.evaluate(X_test_seq, PL_test_seq, verbose=0)

# Predict path loss for the test set
PL_pred = model.predict(X_test_seq).flatten()

# Calculate additional metrics
rmse_test = np.sqrt(mean_squared_error(PL_test_seq, PL_pred))
r2_test = r2_score(PL_test_seq, PL_pred)
mape_test = mean_absolute_percentage_error(PL_test_seq, PL_pred)
median_ae_test = median_absolute_error(PL_test_seq, PL_pred)

# Create a results DataFrame
results = pd.DataFrame({
    'Metric': ['Training Loss (MSE)', 'Training MAE',
               'Test Loss (MSE)', 'Test MAE',
               'Test RMSE', 'R2 Score',
               'Test MAPE (%)', 'Test Median AE'],
    'Value': [train_loss, train_mae,
              test_loss, test_mae,
              rmse_test, r2_test,
              mape_test * 100, median_ae_test]
})

# Display the results table
print("\nModel Evaluation Metrics:")
display(results)

In [ ]:
# Visualize training history and model predictions

# Plot training and validation loss
plt.figure(figsize=(18, 5))

# Plot Loss
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss (MSE)')
plt.plot(history.history['val_loss'], label='Validation Loss (MSE)')
plt.xlabel('Epochs')
plt.ylabel('Mean Squared Error')
plt.title('Training and Validation Loss')
plt.legend()
plt.grid(True)

# Plot MAE
plt.subplot(1, 2, 2)
plt.plot(history.history['mae'], label='Training MAE')
plt.plot(history.history['val_mae'], label='Validation MAE')
plt.xlabel('Epochs')
plt.ylabel('Mean Absolute Error')
plt.title('Training and Validation MAE')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

# Plot predicted vs actual values
plt.figure(figsize=(12, 4))
sns.scatterplot(x=PL_test_seq, y=PL_pred, alpha=0.6, edgecolor='w', s=70)
plt.plot([PL_test_seq.min(), PL_test_seq.max()], [PL_test_seq.min(), PL_test_seq.max()], 'r--', lw=2)
plt.xlabel('Actual Path Loss')
plt.ylabel('Predicted Path Loss')
plt.title('Actual vs Predicted Path Loss')
plt.grid(True)
plt.show()

In [ ]:
# Perform time-aware cross-validation to assess model robustness and display results in a table

seq_len = X_train_seq.shape[1]
n_features = X_train_seq.shape[2]

fold = 1
cv_results_list = []

for train_index, val_index in ps_seq.split():
    print(f"Training fold {fold}...")

    # Split data into training and validation for the current fold
    X_train_fold, X_val_fold = X_train_seq[train_index], X_train_seq[val_index]
    PL_train_fold, PL_val_fold = PL_train_seq[train_index], PL_train_seq[val_index]

    # Create a new instance of the model for each fold
    model = create_rnn_model(seq_len, n_features)

    # Define Callbacks
    early_stop_cv = EarlyStopping(
        monitor='val_loss',
        patience=30,
        restore_best_weights=True
    )

    reduce_lr_cv = ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=10,
        min_lr=1e-6,
        verbose=0
    )

    # Train the model on the current fold
    history_cv = model.fit(
        X_train_fold,
        PL_train_fold,
        validation_data=(X_val_fold, PL_val_fold),
        epochs=500,
        batch_size=128,
        callbacks=[early_stop_cv, reduce_lr_cv],
        verbose=0,
        shuffle=False
    )

    # Predict path loss for the test set
    PL_pred_cv = model.predict(X_test_seq).flatten()

    # Calculate metrics
    test_loss, test_mae = model.evaluate(X_test_seq, PL_test_seq, verbose=0)
    rmse_cv = np.sqrt(mean_squared_error(PL_test_seq, PL_pred_cv))
    r2_cv = r2_score(PL_test_seq, PL_pred_cv)
    mape_cv = mean_absolute_percentage_error(PL_test_seq, PL_pred_cv)
    median_ae_cv = median_absolute_error(PL_test_seq, PL_pred_cv)

    # Append metrics to the results list
    cv_results_list.append({
        'Fold': fold,
        'Test Loss (MSE)': round(test_loss, 4),
        'Test MAE': round(test_mae, 4),
        'Test RMSE': round(rmse_cv, 4),
        'R2 Score': round(r2_cv, 4),
        'Test MAPE (%)': round(mape_cv * 100, 2),
        'Test Median AE': round(median_ae_cv, 4)
    })

    print(
        f"Fold {fold} - Test Loss (MSE): {test_loss:.4f}, Test MAE: {test_mae:.4f}, "
        f"RMSE: {rmse_cv:.4f}, R2: {r2_cv:.4f}, MAPE: {mape_cv*100:.2f}%, "
        f"Median AE: {median_ae_cv:.4f}"
    )
    fold += 1

# Create a DataFrame from the results
cv_results_df = pd.DataFrame(cv_results_list)

# Calculate average and standard deviation for each metric
cv_summary = cv_results_df.agg(['mean', 'std']).round(4).reset_index()
cv_summary.rename(columns={'index': 'Metric'}, inplace=True)

# Display Cross-Validation Results
print("Time-Aware Cross-Validation Results:")
display(cv_results_df)

print("Cross-Validation Summary:")
display(cv_summary)